# LCEL: Agent 核心组件
![Agent](pngs/LCELAgents.png)
![Agent](pngs/LCELAgentsCore.png)

In [19]:
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI
from langchain.tools import tool

# LCEL Agent 是 LangChain 中的一种核心组件，用于处理复杂的任务分解和执行。
# 它通过结合语言模型和工具（如搜索引擎、计算器等）来完成多步骤任务。
# LCEL Agent 的主要特点是其灵活性和可扩展性，适用于各种场景。


# 定义一个简单的工具
@tool
def add_numbers(input: str) -> str:
  """接受两个数字并返回它们的和"""
  numbers = list(map(int, input.split()))
  return str(sum(numbers))

# 初始化语言模型
llm = OpenAI(temperature=0)

# 定义工具列表
tools = [Tool(name="Addition", func=add_numbers, description="用于计算两个数字的和")]

# 初始化 LCEL Agent
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

# 使用 Agent 执行任务
result = agent.run("请帮我计算 12 和 8 的和。")
print(result)

ValidationError: 1 validation error for OpenAI
  Value error, Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. [type=value_error, input_value={'temperature': 0, 'model...ne, 'http_client': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

### 一个比较复杂的Agent
- RAG
- 搜索

In [20]:
from langchain.pydantic_v1 import  BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from langchain_community.utilities import SerpAPIWrapper
import sys
import os

# Add the root directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from bocha_api import bocha_websearch_tool

print(bocha_websearch_tool("今天的老黄历"))

引用: 1
标题: 今天是什么日子?2025年黄历宜忌每日查询_万年历_财源_农历
URL: https://www.sohu.com/a/880565401_121123713
摘要:  今天是2025年4月7日星期一 2025年黄历万年历 农历三月初十 乙巳蛇年 戊寅月 丙午日 ㊗️财源广进､万事如意 美好的一天,早安好☕️🏃 返回搜狐,查看更多
网站名称: 搜狐
网站图标: https://th.bochaai.com/favicon?domain_url=https://www.sohu.com/a/880565401_121123713
发布时间: 2025-04-07T06:38:00Z

引用: 2
标题: 2025年4月7日黄历:宜嫁娶,忌安葬_红色_建议_颜色
URL: https://www.sohu.com/a/880525229_122001004
摘要: 日期:公历2025年4月7日,星期一,农历三月初八(乙巳年 庚辰月 丙午日) 生肖运势:今日最佳:马､蛇､羊;次吉生肖:猴､鸡､狗;今日带衰:鼠､牛､兔 方位:喜神方位:西南;财神方位:正东;福神方位:东南;阳贵方位:东北 幸运数字:3､8 幸运颜色:红色､黄色､白色 宜忌:宜:结婚､开业､交易､签约､祈福､祭祀;忌:安葬､破土､修造､动土 吉时:今日吉时:巳时(09:00-11:00)､午时(11:00-13:00)､未时(13:00-15:00) 建议:今天是个适合举行婚礼和开业的好日子,尤其是对于属马､蛇､羊的朋友来说,运势特别好｡如果有重要的商业活动或家庭聚会,可以考虑安排在上午9点到下午3点之间进行,这样更有利于事情的顺利开展｡ 择吉:本月适宜嫁娶的日子有4月7日､4月16日､4月25日; 【大吉色】红色:今天穿上红色的衣服,不仅能够提升个人魅力,还能带来好运｡ 【次吉色】黄色:黄色也是今天的幸运色之一,它代表着财富和智慧,适合在工作中穿着,有助于提升工作效率｡ 【平平色】白色:白色虽然不是今天的最佳选择,但也能带来一定的平衡感,适合日常穿搭｡ 【忌用色】绿色:绿色在今天不太吉利,建议避免穿着或使用绿色的物品｡ 温馨小提示:五行穿衣色不仅可以用于衣服,日常生活中随身携带的小物和佩戴的饰品都可以参考五行旺运颜色来搭配哦! 建议:结合本月宜忌和其余信息,合理安排自己的生活和工作,注意

In [24]:
# RAG
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_ollama.embeddings import OllamaEmbeddings

# 使用 WebBaseLoader 加载指定 URL 的文档
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
docs = loader.load()  # 加载文档
print(f"Loaded {len(docs)} documents")  # 打印加载的文档数量

# 使用 RecursiveCharacterTextSplitter 对文档进行分块
documents = RecursiveCharacterTextSplitter(
  chunk_size=1000,  # 每个块的最大字符数
  chunk_overlap=200  # 块之间的重叠字符数
).split_documents(docs)

# 使用 OllamaEmbeddings 创建嵌入，并将文档存储到 FAISS 向量数据库中
vector = FAISS.from_documents(documents, OllamaEmbeddings(model="bge-m3:latest"))

# 将向量数据库转换为检索器
retriever = vector.as_retriever()

# 使用检索器进行查询，并以流式方式返回结果
retriever.stream("How to use the smith platform")


Loaded 1 documents


<generator object Runnable.stream at 0x11c8524d0>

In [25]:
# 把检索器加入到工具中
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
  retriever,
  "langchain_search",
  "搜索有关 LangSmith 的信息。 关于 LangSmith 的任何问题都使用该工具。"
)

retriever_tool

Tool(name='langchain_search', description='搜索有关 LangSmith 的信息。 关于 LangSmith 的任何问题都使用该工具。', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x10a906b60>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x11c820b00>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x10ac85f80>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x11c820b00>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', respon

In [26]:
# 工具集
tools = [bocha_websearch_tool, retriever_tool]
tools

[StructuredTool(name='bocha_websearch_tool', description='使用Bocha Web Search API 进行网页搜索。\n\n参数:\n- query: 搜索关键词\n- freshness: 搜索的时间范围\n- summary: 是否显示文本摘要\n- count: 返回的搜索结果数量\n\n返回:\n- 搜索结果的详细信息，包括网页标题、网页URL、网页摘要、网站名称、网站Icon、网页发布时间等。', args_schema=<class 'langchain_core.utils.pydantic.bocha_websearch_tool'>, func=<function bocha_websearch_tool at 0x10b8fdf80>),
 Tool(name='langchain_search', description='搜索有关 LangSmith 的信息。 关于 LangSmith 的任何问题都使用该工具。', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x10a906b60>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x11c820b00>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_rel

In [27]:
# 定义模型
from langchain_ollama.llms import OllamaLLM
llm = OllamaLLM(model="qwen2.5:7b",temperature=0)

In [28]:
# 从 Hub 中获取模板
from langchain import hub

# 一个最简单的模板，带记忆
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

/opt/homebrew/Caskroom/miniconda/base/envs/rag/lib/python3.13/site-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [35]:
# 创建agent
# from langchain.agents import create_openai_functions_agent
# 没有OpenAIKey
# agent = create_openai_functions_agent(llm, tools, prompt)

from langchain.agents import AgentType, initialize_agent
agent = initialize_agent(
  tools= [retriever_tool],
  llm=llm,
  agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
  verbose=True
)

In [40]:
# 创建Agent 执行器 AgentExecutor
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [38]:
# 执行
agent_executor.invoke({"input":"hi"})



> Entering new AgentExecutor chain...


> Entering new None chain...


TypeError: Agent.plan() got multiple values for argument 'intermediate_steps'